In [ ]:
# data_loader
    # read from model system sub-folders in data
    # write to ../strctrd/
    # read from ../strctrd/ to correction methods, write to results/files

https://pnnl-compbio.github.io/coderdata/pages/usage

#### Download data

In [ ]:
import coderdata as cd

In [ ]:
# cd.download_data_by_prefix('beataml')
# cd.download_data_by_prefix('cell_line')
# cd.download_data_by_prefix('cptac')
# cd.download_data_by_prefix('hcmi')

##### Exacloud download

In [ ]:
pip install coderdata

##### Exacloud upload

In [ ]:
ls ../data/beataml/

#### Set systems

In [ ]:
pwd

In [ ]:
%whos

In [ ]:
import coderdata as cd
cell_line = cd.DatasetLoader('cell_line', data_directory = '../data/cell_line/') # a
cptac = cd.DatasetLoader('cptac', data_directory = '../data/cptac/') # b
beataml = cd.DatasetLoader('beataml', data_directory = '../data/beataml/') # c
hcmi = cd.DatasetLoader('hcmi', data_directory = '../data/hcmi/') # d

print('DatasetLoaders in RAM')
systems = 'cell-line+CPTAC'

In [ ]:
# Four data/<prefix> dirs populated
    # target output dir: strctrd/files
    # record:
        # # target output dir: strctrd/one_cncr

#### Imports and functions

In [ ]:
import glob
import pandas as pd
import umap
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
# Traspose long-form X-file into biomarkers as columns,
def extract(df_lite):
    import time
    start = time.time()
    data_type = df_lite.columns[2] # extract marker values

    dot_T = df_lite.pivot_table(
        index='improve_sample_id',
        columns='entrez_id',
        values=data_type,
        aggfunc='mean'             # average duplicate values
    )

    end = time.time()
    wall_clock = end - start
    return str(round(wall_clock / 60, 2)) + ' minutes', dot_T

In [ ]:
# Extract ids and biomarker values
def df_check(X_n):
    df_lite = X_n.iloc[:, :3] # cut the last two columns, source and study
    size = f"{df_lite.shape[0]:,}"
    na_count = f"{df_lite.isna().sum().sum():,}"
    inf_count = f"{df_lite.isin([np.inf, -np.inf]).sum().sum():,}"
    return df_lite, size, na_count, inf_count

In [ ]:
# dot_T = g(d_typ, dot_T.copy())
def g(d_typ, df):
    """
    Checks the data types of columns and index in a DataFrame and prints informative messages.

    Args:
        df (pandas.DataFrame): The DataFrame to check.

    Returns:
        None
    """

    if df.columns.dtype == 'float64' and df.index.dtype == 'float64':
        print('both float')
        df = float_to_string(d_typ, df)
    elif df.columns.dtype == 'float64' and df.index.dtype == 'int':
        print('columns are float, index are int')
        df = indx_int_colm_flt(d_typ, df)
    elif df.columns.dtype == 'int' and df.index.dtype == 'float':
        print('columns are int, index are float, fail, write another function')
        # forth function
    elif df.columns.dtype == 'int' and df.index.dtype == 'int':
        print('columns are int, index are int')
        df = int_to_string(d_typ, df)
    else:
        print('non int or float dtype detected')
    return df

In [ ]:
def int_to_string(d_typ, dot_T):
    dot_T.columns = dot_T.columns.map(str)
    dot_T.columns = ['entrz_' + d_typ + i for i in dot_T.columns] #
    dot_T.columns.name = 'entrez_id'

    dot_T.index = dot_T.index.map(str)
    dot_T.index = ['smpl_id_' + i for i in dot_T.index]
    dot_T.index.name = 'improve_sample_id'
    return dot_T

In [ ]:
def indx_int_colm_flt(d_typ, dot_T):
    dot_T.columns = dot_T.columns.map(str)
    dot_T.columns = [i.split('.')[0] for i in dot_T.columns]
    dot_T.columns = ['entrz_' + d_typ + i for i in dot_T.columns]
    dot_T.columns.name = 'entrez_id'
    
    dot_T.index = dot_T.index.map(str)
    dot_T.index = ['smpl_id_' + i for i in dot_T.index]
    dot_T.index.name = 'improve_sample_id'
    return dot_T

In [ ]:
def float_to_string(d_typ, dot_T):
    dot_T.columns = dot_T.columns.map(str)
    dot_T.columns = [i.split('.')[0] for i in dot_T.columns]
    dot_T.columns = ['entrz_' + d_typ + i for i in dot_T.columns]
    dot_T.columns.name = 'entrez_id'
    
    dot_T.index = dot_T.index.map(str)
    dot_T.index = [i.split('.')[0] for i in dot_T.index]
    dot_T.index = ['smpl_id_' + i for i in dot_T.index]
    dot_T.index.name = 'improve_sample_id'
    return dot_T

#### Systems toggles

In [ ]:
systems = 'cell-line+CPTAC'

In [ ]:
systems = 'cell-line+BeatAML'

In [ ]:
systems = 'cell-line+CPTAC'

#### Samples, abstract systems to a and b

In [ ]:
# Cell line and CPTAC
sys_a_samp = cell_line.samples
sys_a = 'cell-line'
sys_a_lbl = 'cell_line'

sys_b_samp = cptac.samples
sys_b = 'cptac'
sys_b_lbl = 'CPTAC'

In [ ]:
# AML
sys_a_samp = cell_line.samples
sys_a = 'cell-line'
sys_a_lbl = 'cell_line'

sys_b_samp = beataml.samples
sys_b = 'beataml'
sys_b_lbl = 'BeatAML'

#### Cancer_type view

In [ ]:
print(sys_a, 'is system a')
sys_a_samp.cancer_type.value_counts()[:60]

In [ ]:
print(sys_b, 'is system b')
sys_b_samp.cancer_type.value_counts()[:50]

In [ ]:
print(sys_c, 'is system c')
sys_c_samp.cancer_type.value_counts()[:2] # hold for 3-project combos

#### Cancer_type toggles, cell line as a, CPTAC as b

In [ ]:
##### single cancers to /strctrd/one_cncr

In [ ]:
# Lung adeno
cncr = 'lung-adeno'
cncr_lbl = 'lung_adeno'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Lung Adenocarcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Lung adenocarcinoma'] # cp

In [ ]:
# Pancreatic adeno
cncr = 'pancreatic-adeno'
cncr_lbl = 'pancreatic_adeno'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Pancreatic Adenocarcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Pancreatic ductal adenocarcinoma'] # cp

In [ ]:
# Head and neck - no prot samples?, 5 trns samples
cncr = 'head-neck'
cncr_lbl = 'head_neck'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Head and Neck Squamous Cell Carcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Head and Neck squamous cell carcinoma'] # cp

In [ ]:
# Colon adeno
cncr = 'colon-adeno'
cncr_lbl = 'colon_adeno'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Colon Adenocarcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Colon adenocarcinoma'] # cp

In [ ]:
# Glioblastoma
cncr = 'glioblastoma'
cncr_lbl = 'glioblastoma'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Glioblastoma']
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Glioblastoma multiforme'] # cp

In [ ]:
# Renal clear cell
cncr = 'renal-clear-cell'
cncr_lbl = 'renal_clear_cell'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Renal Clear Cell Carcinoma']
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Clear cell renal cell carcinoma'] # cp

#### BRCA

In [ ]:
# Breast Ductal - 3 transciptomic samples for cell line
cncr = 'breast-ductal'
cncr_lbl = 'breast_ductal'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Breast Invasive Ductal Carcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Breast carcinoma']

In [ ]:
# Breast Lobular - # one cell line sample for prot, 8 samples trans
cncr = 'breast-lobular'
cncr_lbl = 'breast_lobular'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Breast Invasive Lobular Carcinoma'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Breast carcinoma']

In [ ]:
# Breast NOS - # two cell line samples for prot - 3 transciptomic samples for cell line
cncr = 'breast-nos'
cncr_lbl = 'breast_nos'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Breast Invasive Carcinoma, NOS'] # cl
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'Breast carcinoma']

##### brca single cancers to /strctrd/one_cncr

In [ ]:
pwd

In [ ]:
ls

In [ ]:
# strctr_one.py
# Script to build one_cncr files
    # Sript for building strctrd/two_cncr files
    # will read from strctrd/one_cncr files
# pwd: /mdl-sys-bnchmrk/code

# Imports
import coderdata as cd
import glob
import pandas as pd
import umap
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# pwd
# mdl-sys-bnchmrk/code

# ls
# strctr.py

# Import functions
import strctr

systems = 'cell-line+CPTAC'

# Read data (construct loader objects)
cell_line = cd.DatasetLoader('cell_line', data_directory = '../data/cell_line/') # a
cptac = cd.DatasetLoader('cptac', data_directory = '../data/cptac/') # b

# Cell lines are system A
sys_a_samp = cell_line.samples
sys_a = 'cell-line'
sys_a_lbl = 'cell_line'

# CPTAC is system B
sys_b_samp = cptac.samples
sys_b = 'cptac'
sys_b_lbl = 'CPTAC'

# BeatAML is system B
#

# Transcriptomics data modality extraction
modality = 'transcriptomics' # to file name
moda = 'tran_' # to columns and index

# Proteomics data modality extraction
# modality = 'proteomics' # to file name
# moda = 'prot_' # to columns and index
## mda_n_sys_a = cell_line.proteomics[cell_line.proteomics.improve_sample_id.isin(ids_proj_a)] # cl
## mda_n_sys_b = cptac.proteomics[cptac.proteomics.improve_sample_id.isin(ids_proj_b)]

# Copy number and Mutations data modality extractions insertion point

names = ['breast-ductal', 'breast-lobular', 'breast-nos']
labels = ['breast_ductal', 'breast_lobular', 'breast_nos']
a_list = ['Breast Invasive Ductal Carcinoma',
          'Breast Invasive Lobular Carcinoma',
          'Breast Invasive Carcinoma, NOS']
b_list = ['Breast carcinoma',
          'Breast carcinoma',
          'Breast carcinoma']

for i, cncr in enumerate(names):
    # cncr = a_list[i]
    cncr_lbl = labels[i]
    
    sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == a_list[i]]
    sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == b_list[i]]

    ids_sys_a = sys_a_samp_canc_n.improve_sample_id # cl
    ids_sys_b = sys_b_samp_canc_n.improve_sample_id # cp

    mda_n_sys_a = cell_line.transcriptomics[cell_line.transcriptomics.improve_sample_id.isin(ids_sys_a)] # cl
    mda_n_sys_b = cptac.transcriptomics[cptac.transcriptomics.improve_sample_id.isin(ids_sys_b)]
    break
    df_lite, size, na_count, inf_count = df_check(mda_n_sys_a)
    print(sys_a, '| sys a')
    print(cncr, modality)
    print('len: ', size)
    print('NaNs: ', na_count)
    print('Infs: ', inf_count)

    wall_clock, dot_T = extract(df_lite)
    # print(wall_clock)
    # dot_T.iloc[:3, :3]
    #
    # dot_T = g(moda, dot_T)
    dot_T = g(moda, dot_T.copy())
    dot_T.dropna(axis = 1, inplace = True)
    a = dot_T # cell line

    df_lite, size, na_count, inf_count = df_check(mda_n_sys_b)
    print(sys_b, '| sys b')
    print(cncr, modality)
    print('len: ', size)
    print('NaNs: ', na_count)
    print('Infs: ', inf_count)
    
    wall_clock, dot_T = extract(df_lite)
    dot_T = g(moda, dot_T.copy())
    dot_T.dropna(axis = 1, inplace = True)
    b = dot_T # cptac
    
    a.insert(0, 'Cancer_type', cncr_lbl)
    b.insert(0, 'Cancer_type', cncr_lbl)
    a.insert(0, 'System', sys_a_lbl)
    b.insert(0, 'System', sys_b_lbl)

    ab = pd.concat([a, b], axis=0, join='inner')
    print(ab.System.value_counts())
    print(ab.Cancer_type.value_counts())

    out_one = '../strctrd/one_cncr/'

    # Write two-system, single cancer type to disk
    ab.to_csv(
        '../strctrd/'+out_one+'/'+cncr+'_'+modality+'_'+systems+'.tsv',
        sep = '\t')
    # break

##### Merged BRCA as single cancer

##### intertools, brca, two cancer

In [ ]:
from itertools import combinations

#### BeatAML devel

In [ ]:
cncr = 'beat-aml'
cncr_lbl = 'beat_aml'
sys_a_samp_canc_n = sys_a_samp[sys_a_samp.cancer_type == 'Acute Myeloid Leukemia']
sys_b_samp_canc_n = sys_b_samp[sys_b_samp.cancer_type == 'ACUTE MYELOID LEUKAEMIA']

In [ ]:
# beat aml as single cancer


#### Extract sample ids

In [ ]:
print(cncr, sys_a, sys_b)
ids_sys_a = sys_a_samp_canc_n.improve_sample_id # cl
ids_sys_b = sys_b_samp_canc_n.improve_sample_id # 

#### Data extraction, CPTAC

##### trns

In [ ]:
print(cncr)
modality = 'transcriptomics' # to file name # BRCA bug?
moda = 'tran_' # to columns and index
mda_n_sys_a = cell_line.transcriptomics[cell_line.transcriptomics.improve_sample_id.isin(ids_sys_a)] # cl
mda_n_sys_b = cptac.transcriptomics[cptac.transcriptomics.improve_sample_id.isin(ids_sys_b)]

##### prot

In [ ]:
modality = 'proteomics' # to file name
moda = 'prot_' # to columns and index
mda_n_sys_a = cell_line.proteomics[cell_line.proteomics.improve_sample_id.isin(ids_sys_a)] # cl
mda_n_sys_b = cptac.proteomics[cptac.proteomics.improve_sample_id.isin(ids_sys_b)]

##### cn

In [ ]:
modality = 'copy-number' # to file name
moda = 'cpnm_' # to columns and index
mda_n_sys_a= cell_line.copy_number[cell_line.copy_number.improve_sample_id.isin(ids_sys_a)]
mda_n_sys_a= cell_line.copy_number[cptac.copy_number.improve_sample_id.isin(ids_sys_b)]

In [ ]:
modality = 'mutations' # to file name
moda = 'muta_' # to columns and index
mda_n_sys_a = cell_line.mutations[cell_line.mutations.improve_sample_id.isin(ids_sys_a)]
mda_n_sys_b = cptac.mutations[cptac.mutations.improve_sample_id.isin(ids_sys_b)]

#### Data extraction, AML

##### trans

In [ ]:
print(cncr)
modality = 'transcriptomics' # to file name # BRCA bug?
moda = 'tran_' # to columns and index
mda_n_sys_a = cell_line.transcriptomics[cell_line.transcriptomics.improve_sample_id.isin(ids_sys_a)] # cl
mda_n_sys_b = beataml.transcriptomics[beataml.transcriptomics.improve_sample_id.isin(ids_sys_b)]

##### prot

In [ ]:
print(cncr)
modality = 'proteomics' # to file name
moda = 'prot_' # to columns and index
mda_n_sys_a = cell_line.proteomics[cell_line.proteomics.improve_sample_id.isin(ids_sys_a)] # cl
mda_n_sys_b = beataml.proteomics[beataml.proteomics.improve_sample_id.isin(ids_sys_b)]

#### Project A, cell line

In [ ]:
df_lite, size, na_count, inf_count = df_check(mda_n_sys_a)
print(sys_a, '| sys a')                 #               ^ 
print(cncr, modality)
print(size)
print(na_count)
print(inf_count)

wall_clock, dot_T = extract(df_lite)
dot_T = g(moda, dot_T.copy())
dot_T.dropna(axis = 1, inplace = True)
# add zero check
a = dot_T # cell line

##### a devel

In [ ]:
wall_clock, dot_T = extract(df_lite)
print(wall_clock)
dot_T.iloc[:3, :3]

In [ ]:
cncr

In [ ]:
moda

In [ ]:
# dot_T = g(d_typ, dot_T.copy())
dot_T = g(moda, dot_T.copy())
dot_T.iloc[:3, :3]

In [ ]:
dot_T.shape

In [ ]:
dot_T.dropna(axis = 1, inplace = True)

In [ ]:
dot_T.shape

In [ ]:
a = dot_T # cell line

#### Project B, CPTAC

In [ ]:
df_lite, size, na_count, inf_count = df_check(mda_n_sys_b)
print(sys_b, '| sys b')                 #               ^ 
print(cncr, modality)
print('len: ', size)
print('NaNs: ', na_count)
print('Infs: ', inf_count)

wall_clock, dot_T = extract(df_lite)
dot_T = g(moda, dot_T.copy())
dot_T.dropna(axis = 1, inplace = True)
b = dot_T # 

##### b devel

In [ ]:
wall_clock, dot_T = extract(df_lite)
print(wall_clock)
dot_T.iloc[:3, :3]

In [ ]:
dot_T = g(moda, dot_T.copy())

In [ ]:
dot_T.iloc[:3, :3]

In [ ]:
dot_T.shape

In [ ]:
dot_T.dropna(axis = 1, inplace = True)

In [ ]:
dot_T.shape

In [ ]:
b = dot_T # cptac

#### Labels

##### Check a and b are different

In [ ]:
a.iloc[:3, :3]

In [ ]:
b.iloc[:3, :3]

##### Continue

In [ ]:
a.insert(0, 'Cancer_type', cncr_lbl)
b.insert(0, 'Cancer_type', cncr_lbl)
a.insert(0, 'System', sys_a_lbl)
b.insert(0, 'System', sys_b_lbl)

In [ ]:
ab = pd.concat([a, b], axis=0, join='inner')

In [ ]:
print(cncr, modality)
ab

In [ ]:
ab.System.value_counts()

In [ ]:
ab.Cancer_type.value_counts()

##### Set out_dir for one_cncr or two_cncr

In [ ]:
ls ../strctrd/one_cncr/

In [ ]:
# Set target output dirs
out_2 = '../strctrd/two_cncr/' # This means encoding two cancers at the same time
                                    # Read from out_1 to build ^
out_1 = '../strctrd/one_cncr/' # This means encoding one cancer at a time

In [ ]:
out_dir = out_1
# out_dir = out_2

##### continue

In [ ]:
# out, u_o, cancer, data_type
out_dir, cncr, modality

In [ ]:
# Write two-system, single cancer type to disk
ab.to_csv(out_dir+
    '/'+cncr+'_'+modality+'_'+systems+'.tsv',
    sep = '\t')

In [ ]:
chk_ab = pd.read_csv(out_dir+
    '/'+cncr+'_'+modality+'_'+systems+'.tsv',
    sep = '\t', index_col = 0)

In [ ]:
print(out_dir, cncr, modality, systems)
chk_ab

In [ ]:
ls '../strctrd/one_cncr/'

In [ ]:
len(glob.glob('../strctrd/one_cncr/*.tsv'))

In [ ]:
ls '../strctrd/one_cncr/1_run_1/'

##### Check proteomics negatives

In [ ]:
prot_neg_fls = glob.glob('../strctrd/one_cncr/*proteomics_cell-line+CPTAC.tsv')

In [ ]:
prot_neg_fls

In [ ]:
for png in prot_neg_fls:
    chk_ab = pd.read_csv(png,
        sep = '\t', index_col = 0)
    print(png.split('/')[3])
    print(chk_ab.iloc[:, 2:].min().min())
    print(' ')
    # break

In [ ]:
chk_ab.iloc[:, 2:].min().min()

#### Dbl Cncr

In [ ]:
print(cncr)
ab.iloc[:3, :3]

In [ ]:
ab.shape

#### Disk read Cancer_n

In [ ]:
# Interactive zone for batch correction experiments

In [ ]:
# Template read-in path
output/sng_cncr/lung_adeno_transcriptomics_cell_line_CPTAC.tsv

#### Experimental record

In [ ]:
# toggle zone 1

In [ ]:
systems = 'cell-line+CPTAC'

In [ ]:
cncr_n = 'renal-clear-cell'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [ ]:
cncr_n = 'pancreatic-adeno'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [ ]:
cncr_n = 'lung-adeno'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [ ]:
cncr_n = 'glioblastoma'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [ ]:
cncr_n = 'colon-adeno'
moda_n = 'transcriptomics'
sys_n = 'cell-line+CPTAC'

In [ ]:
# toggle zone 2

In [ ]:
c1 = pd.read_csv( # single cancer 1
    '../output/sng-cncr/'+cncr_n+'_'+moda_n+'_'+sys_n+'.tsv',
    sep = '\t', index_col = 0)

In [ ]:
c1.iloc[:3, :3]

In [ ]:
c2 = pd.read_csv( # single cancer 1
    '../output/sng-cncr/'+cncr_n+'_'+moda_n+'_'+sys_n+'.tsv',
    sep = '\t', index_col = 0)

In [ ]:
c2.iloc[:3, :3]

In [ ]:
print(moda_n)
dbl_cncr = pd.concat([c1, c2], axis = 0, join = 'inner')
dbl_cncr.shape

In [ ]:
dbl_cncr.iloc[:3, :3]

In [ ]:
dbl_cncr.iloc[-3:, :3]

In [ ]:
# two sets of number for each label type

In [ ]:
dbl_cncr.System.value_counts()

In [ ]:
dbl_cncr.Cancer_type.value_counts()

In [ ]:
out_dbl, cncr, cncr_n, moda_n, systems

In [ ]:
ls ../output/dbl-cncr/

In [ ]:
dbl_cncr.to_csv('../output/'+out_dbl+'/'+cncr+'+'+cncr_n+'_'+moda_n+'_'+systems+'.tsv',
                    sep = '\t')

In [ ]:
dbl_chk = pd.read_csv('../output/'+out_dbl+'/'+cncr+'+'+cncr_n+'_'+moda_n+'_'+systems+'.tsv',
                    sep = '\t', index_col = 0)

In [ ]:
dbl_chk